In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import sklearn.preprocessing as pp
import sklearn.model_selection as ms
import sklearn.metrics as metrics
import json

### Load Dataset

In [5]:
df = pd.read_csv('./dataset/Train_Data.csv')
sdf = pd.read_csv('./dataset/Test_Data.csv')
sample = pd.read_csv('./dataset/Sample_Submission_File.csv')
df = df.set_index('Id')
sdf = sdf.set_index('Id')
df.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
price = df.SalePrice
print("Average sale price: " + "${:,.0f}".format(price.mean()))

Average sale price: $181,253


### Combine test and train for preprocessing

In [7]:
df = df.drop('SalePrice', axis=1)
all_df = df.append(sdf)
all_df.shape


(1460, 79)

In [8]:
all_features = 'MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition'.split(',')
numeric_features = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','TotalBsmtSF','Fireplaces', 'GarageCars', 'GarageArea','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
categorical_features = [f for f in all_features if not(f in numeric_features)]

(len(all_features), len(categorical_features), len(numeric_features))


(79, 53, 26)

In [9]:
numeric_df = all_df[numeric_features]
numeric_df.shape


(1460, 26)

### 
- Impute Only impute for numeric variables, categorical will have a NA column


In [10]:
X = numeric_df.as_matrix()

# Impute missing

imp = pp.Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imp = imp.fit(X)
X = imp.transform(X)
X.shape

/home/macab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/macab/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


(1460, 26)

In [11]:
scaler = pp.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X[0, :]

array([-0.14618854, -0.20714171,  0.51410389,  0.57542484, -0.28865283,
       -0.94459061, -0.12024172,  0.37033344,  1.10781015, -0.24106104,
        0.78974052,  1.22758538,  0.16377912, -0.21145358,  0.91220977,
       -0.45930254, -0.95122649,  0.31172464,  0.35100032, -0.75217584,
        0.21650316, -0.3593249 , -0.11633929, -0.27020835, -0.06869175,
       -0.08768781])

### Expand Categorical Cols

In [12]:
def process_categorical(ndf, df, categorical_features):
    for f in categorical_features:
        new_cols = pd.DataFrame(pd.get_dummies(df[f]))
        new_cols.index = df.index
        ndf = pd.merge(ndf, new_cols, how = 'inner', left_index=True, right_index=True)
    return ndf

numeric_df = pd.DataFrame(X)
numeric_df.index = all_df.index
combined_df = process_categorical(numeric_df, all_df, categorical_features)
combined_df.head()



,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,ConLw,New,Oth,WD,Abnorml,AdjLand,Alloca,Family,Normal,Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,-0.146189,-0.207142,0.514104,0.575425,-0.288653,-0.944591,-0.120242,0.370333,1.107810,-0.241061,...,0,0,0,1,0,0,0,0,1,0
2,0.524992,-0.091886,-0.570750,1.171992,-0.288653,-0.641228,-0.120242,-0.482512,-0.819964,3.948809,...,0,0,0,1,0,0,0,0,1,0
3,-0.011953,0.073480,0.325915,0.092907,-0.288653,-0.301643,-0.120242,0.515013,1.107810,-0.241061,...,0,0,0,1,0,0,0,0,1,0
4,-0.369915,-0.096897,-0.570750,-0.499274,-0.288653,-0.061670,-0.120242,0.383659,1.107810,-0.241061,...,0,0,0,1,1,0,0,0,0,0
5,0.703973,0.375148,1.366489,0.463568,-0.288653,-0.174865,-0.120242,1.299326,1.107810,-0.241061,...,0,0,0,1,0,0,0,0,1,0


In [13]:


X = combined_df.values
X.shape



(1460, 1769)

In [14]:
#PCA
from sklearn.decomposition import PCA

test_n = df.shape[0]
print(test_n)
pca = PCA()
pca.fit(X[:test_n,:], price)
X = pca.transform(X)
X.shape

1100


(1460, 1100)

In [15]:
X_train = X[:test_n,:]
X_train, X_val, y_train, y_val = ms.train_test_split(X_train, price, test_size=0.3, random_state=0)
X_test = X[test_n:,:]

(X_train.shape, X_val.shape, X_test.shape)

((770, 1100), (330, 1100), (360, 1100))

### Linear Regression Model

In [16]:
from sklearn import linear_model

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

def print_score(alg, score, params):
    print('%s score is %f with params %s' % (alg, score, json.dumps(params)))


### Lasso Regression

In [17]:
for a in np.arange(151., 152., 0.1):
    lasso = linear_model.Lasso(alpha=a, max_iter=2000)
    lasso.fit(X_train, y_train)
    print_score('Lasso', lasso.score(X_val, y_val), {'alpha': a})



Lasso score is 0.786699 with params {"alpha": 151.0}
Lasso score is 0.786696 with params {"alpha": 151.1}
Lasso score is 0.786692 with params {"alpha": 151.2}
Lasso score is 0.786689 with params {"alpha": 151.29999999999998}
Lasso score is 0.786685 with params {"alpha": 151.39999999999998}
Lasso score is 0.786682 with params {"alpha": 151.49999999999997}
Lasso score is 0.786678 with params {"alpha": 151.59999999999997}
Lasso score is 0.786675 with params {"alpha": 151.69999999999996}
Lasso score is 0.786671 with params {"alpha": 151.79999999999995}
Lasso score is 0.786667 with params {"alpha": 151.89999999999995}


In [18]:
lasso = linear_model.Lasso(alpha=151.7, max_iter=2000)
lasso.fit(X_train, y_train)

Lasso(alpha=151.7, copy_X=True, fit_intercept=True, max_iter=2000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

### Ridge Regression


In [19]:
for a in np.arange(25., 26., 0.05):
    ridge = linear_model.Ridge(alpha=a, max_iter=2000)
    ridge.fit(X_train, y_train)
    print_score('Lasso', ridge.score(X_val, y_val), {'alpha': a})

Lasso score is 0.790209 with params {"alpha": 25.0}
Lasso score is 0.790209 with params {"alpha": 25.05}
Lasso score is 0.790208 with params {"alpha": 25.1}
Lasso score is 0.790208 with params {"alpha": 25.150000000000002}
Lasso score is 0.790208 with params {"alpha": 25.200000000000003}
Lasso score is 0.790208 with params {"alpha": 25.250000000000004}
Lasso score is 0.790208 with params {"alpha": 25.300000000000004}
Lasso score is 0.790207 with params {"alpha": 25.350000000000005}
Lasso score is 0.790207 with params {"alpha": 25.400000000000006}
Lasso score is 0.790207 with params {"alpha": 25.450000000000006}
Lasso score is 0.790207 with params {"alpha": 25.500000000000007}
Lasso score is 0.790207 with params {"alpha": 25.550000000000008}
Lasso score is 0.790207 with params {"alpha": 25.60000000000001}
Lasso score is 0.790206 with params {"alpha": 25.65000000000001}
Lasso score is 0.790206 with params {"alpha": 25.70000000000001}
Lasso score is 0.790206 with params {"alpha": 25.75000

In [20]:
ridge = linear_model.Ridge(alpha=25.5, max_iter=2000)
ridge.fit(X_train, y_train)

Ridge(alpha=25.5, copy_X=True, fit_intercept=True, max_iter=2000,
      normalize=False, random_state=None, solver='auto', tol=0.001)

### Gradient Boosting


In [21]:
import xgboost as xgb
params = {'eval_metric':'rmse'}
xm = xgb.DMatrix(X_train, label=y_train)
xmodel = xgb.train(params, xm)
xg_y_pred = xmodel.predict(xgb.DMatrix(X_val))

/home/macab/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [22]:
print('XGBoost score is %f' % metrics.r2_score(y_val, xg_y_pred))
print('Ridge score is %f' % ridge.score(X_val, y_val))
print('Lasso score is %f' % lasso.score(X_val, y_val))
print('Linear Regression score is %f' % lr.score(X_val, y_val))

best = ridge

XGBoost score is 0.807777
Ridge score is 0.790207
Lasso score is 0.786675
Linear Regression score is 0.732606


In [23]:
best.fit(X[:test_n, :], price)
y_submit = best.predict(X_test)
y_submit[y_submit < 0] = 1.
sdf['SalePrice'] = y_submit
sdf.to_csv('submission1.csv', columns = ['SalePrice'])

xmodel = xgb.train(params, xgb.DMatrix(X[:test_n, :], label=price))
y_submit = xmodel.predict(xgb.DMatrix(X_test))
y_submit[y_submit < 0] = 1.
sdf['SalePrice'] = y_submit
sdf.to_csv('submission2.csv', columns = ['SalePrice'])

/home/macab/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/macab/.local/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [24]:
X_train.shape

(770, 1100)